In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, classification_report, accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier

In [2]:
train_df = pd.read_csv("C:/Users/Smart/Desktop/Econ 213/train.csv")
test_df = pd.read_csv("C:/Users/Smart/Desktop/Econ 213/test.csv")

Drop PassID, Ticket, and Cabin for missing variables.

In [3]:
test_ids = pd.DataFrame()
test_ids = test_df['PassengerId']

In [4]:
train_df = train_df.drop(['PassengerId', 'Ticket', 'Cabin'], axis = 1)
test_df = test_df.drop(['PassengerId', 'Ticket', 'Cabin'], axis = 1)

In [5]:
train_df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

Fill in Embarked missing values with most frequent value.

In [6]:
common_port = train_df.Embarked.dropna().mode()[0]
common_port
common_port1 = test_df.Embarked.dropna().mode()[0]
common_port1

'S'

In [7]:
train_df['Embarked'] = train_df['Embarked'].fillna(common_port)
test_df['Embarked'] = test_df['Embarked'].fillna(common_port1)

Fill missing age values with the mean.

In [8]:
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
test_df['Age'] = test_df['Age'].fillna(train_df['Age'].mean())

In [9]:
test_df['Fare'] = test_df['Fare'].fillna(train_df['Fare'].mean())

Make sex a dummy variable.

In [10]:
train_df["Male"] = train_df.Sex.map({"male": 1, "female": 0}).astype(int)
test_df["Male"] = test_df.Sex.map({"male": 1, "female": 0}).astype(int)


In [11]:
train_df['Embarked'] = train_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
test_df['Embarked'] = test_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

Take a look at Survival pitted against other variables to get a look at what variables are most pertinent.

In [12]:
#survived on other features
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [13]:
train_df[['Male', 'Survived']].groupby(['Male'], as_index=False).mean().sort_values(by='Survived', ascending=False)
#meswit dis

,Male,Survived
0,0,0.742038
1,1,0.188908


In [14]:
train_df = train_df.drop(['Sex'], axis=1)
test_df = test_df.drop(['Sex'], axis=1)

In [15]:
test_df = test_df.drop(['Name'], axis=1)

In [16]:
target = train_df.Survived

In [17]:
train_df = train_df.drop(['Survived', 'Name'], axis=1)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(train_df, target, test_size=0.2)

# Logistic Regression

In [20]:
clf = linear_model.LogisticRegression(penalty='l2')
clf.fit(X_train, y_train)
train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)
print("Train acc: {}".format(accuracy_score(y_train, train_predictions)))
print("Test acc: {}".format(accuracy_score(y_test, test_predictions)))

Train acc: 0.7935393258426966
Test acc: 0.8156424581005587


In [21]:
log_preds = clf.predict(test_df)

In [22]:
my_submission = pd.DataFrame({'PassengerId': test_ids, 'Survived': log_preds})

my_submission.to_csv('titanic1.csv', index=False)

This model happened to be my best one.

# K-Nearest Neighbors

In [29]:
clf = KNeighborsClassifier()
gridsearch = GridSearchCV(clf, {"n_neighbors": [1, 3, 5, 7, 9, 11], 'p': [1, 2, 3]}, scoring='f1')
gridsearch.fit(X_train, y_train)
print("Best Params: {}".format(gridsearch.best_params_))
y_pred_train = gridsearch.predict(X_train)
print("Train F1: {}".format(f1_score(y_train, y_pred_train)))
print("Test Classification Report:")
y_pred_test = gridsearch.predict(X_test)
print(classification_report(y_test, y_pred_test))
print("Train Accuracy: {}\tTest accuracy: {}".format(accuracy_score(y_train, y_pred_train),
                                                     accuracy_score(y_test, y_pred_test)))

Best Params: {'n_neighbors': 5, 'p': 1}
Train F1: 0.764
Test Classification Report:
             precision    recall  f1-score   support

          0       0.74      0.79      0.77       106
          1       0.67      0.60      0.63        73

avg / total       0.71      0.72      0.71       179

Train Accuracy: 0.8342696629213483	Test accuracy: 0.7150837988826816


In [30]:
neigh_preds = gridsearch.predict(test_df)

In [31]:
my_submission = pd.DataFrame({'PassengerId': test_ids, 'Survived': neigh_preds})

my_submission.to_csv('titanic2.csv', index=False)

This model just seemed to not perform as well regarding accuracy, recall, and precision.  Worst performing model.

# Random Forest

In [26]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
test_predictions = clf.predict(X_test)
print("Test Classification Report:")
print(classification_report(y_test, test_predictions))

Test Classification Report:
             precision    recall  f1-score   support

          0       0.81      0.85      0.83       106
          1       0.76      0.71      0.74        73

avg / total       0.79      0.79      0.79       179



In [27]:
forest_preds = gridsearch.predict(test_df)

In [28]:
my_submission = pd.DataFrame({'PassengerId': test_ids, 'Survived': forest_preds})

my_submission.to_csv('titanic3.csv', index=False)

Improved in this model on recall, precision and f1 score bubt was still not the best performing model.